In [0]:
df = spark.sql(
"SELECT * FROM sleeper.gold_power_rankings WHERE _matchup_week = 14"
)

In [0]:
display(df)

In [0]:
from pyspark.sql.functions import col, when
import json

# Step 2: For each row, parse fields exactly as written
for row in df.collect():
    # Example: List top scorers exactly as found
    player_lines = []
    for player in row["highest_scoring_players"]:
        player_lines.append(f"{player['highest_scoring_player_name']} ({round(player['highest_scoring_player_points'], 2)})")

    # Example: Bench swap critique only if present
    bench_swaps = row["bench_better_than_starters"] if row["bench_better_than_starters"] else []
    bench_lines = []

    if len(bench_swaps) > 0:
        for swap in bench_swaps:
            bench_lines.append(f"Benched {swap['benched_player_name']} ({round(swap['benched_player_points'],2)}) over {swap['starter_player_name']} ({round(swap['starter_player_points'],2)}) and could've added {round(swap['point_opportunity_cost'],2)} pts")
    else:
        bench_lines.append("None. They played a perfect lineup with the roster they have.")

    decision_impact = (
        "They won their matchup this week reguardless" if row["couldve_won_with_missed_bench_points"] == None 
        else "Their decisions didnt matter as they wouldve lost reguardless" if row["couldve_won_with_missed_bench_points"] == False 
        else f"If they played these players they wouldve won!"
    )

    # Step 6: Format header with ranking and emoji
    if row["power_rank_change"] > 0:
        emoji = "▲"
    elif row["power_rank_change"] < 0:
        emoji = "▼"
    else:
        emoji = "■"

    header = f"{row['power_ranking']}. {row['team_name']} ({emoji}{row['power_rank_change']:+})"

    # Step 7: Generate data-driven, paragraph-style commentary
    # (This uses only the above variables—never invents anything else.)
    # [Here you'd add logic to string together facts as prose]

    print(header)
    print("Owner:", row["owner_name"])
    print("Matchup Points:", row["matchup_points"])
    print("Top 3 star players:", ", ".join(player_lines))
    if bench_lines:
        print("Bad owner decisions:", ", ".join(bench_lines))
    print('Decisions impact: ', decision_impact)
    print('Team Streak:', row['team_streak'])
    print('Record: ', row['team_total_wins'], '-', row['team_total_losses'], '-', row['team_total_ties'])
    print("---")